# Exchanging data among kernels

* **Difficulty level**: easy
* **Time need to lean**: 15 minutes or less
* **Key points**:
  * Passing data across kernels allows you to use the best tool of each language
  * Magic `%get` get variable from another kernel
  * Magic `%put` put variables to another kernel
  * Magic `%with` execute the cell in another kernel with input and output variables  

A SoS notebook can have multiple live kernels with SoS serving as the master kernel to all other kernels (called subkernels). As described in [this tutorial](expand_capture_render.html), SoS can process input and output of subkernels without knowing what they actually do. However, if SoS knows what the kernels do via appropriate language modules, it provides much more powerful ways to communicate with the kernels, the most important of which is the exchange of variables among subkernels.

## Basic concepts

Before we get to the actual magics on how to exchange variables between kernels, it is helpful to understand that, **SoS does not tranfer any variables among kernels, it creates independent homonymous variables of similar types that are native to the destination language**. For example, if you have the following two variables

```R
a = 1
b = c(1, 2)
```

in R and executes a magic 

```Python
%get a b --from R
```
in a SoS cell, SoS actually execute the following statements, in the background, to create variables `a` and `b` in Python

```Python
a = 1
b = [1, 2]
```

As shown in the following figure, language modules try to choose the best method, sometimes in memory and sometimes via disk, to pass variables from one to another kernel, but all the complexity is hidden from you. Variables in different kernels are independent so that changing the value of variables `a` or `b` in one kernel will not affect the variable in another kernel. We also note that `a` and `b` are of different types in Python although they are of the same `numeric` type in `R` (`a` is technically speaking an array of size 1). That is to say, **SoS does not gurantee one to one correspondence between datatypes, and does not gurantee lossless data exchange**.


![user_interface](../media/data_exchange.png)

### Explicit data exchange with magic `%get`

The eastest way to get variable from another kernel is to use magic `%get`. It accepts one or more variable names and an option `--from` if you are not getting from the master `SoS` kernel.

For example, with a variable `data` defined in SoS,

In [1]:
data = [-1, 0, 1, 2, 3]
filename = 'test.txt'

you can `%get` the variables in a R kernel as follows

In [2]:
%get data filename
data

[1] -1  0  1  2  3

In [3]:
filename

[1] "test.txt"

The type of the data is `numeric` because `data` is a numeric list in Python

In [4]:
class(data)

[1] "numeric"

However, if the variable contains different types of data, for example integer and string,

In [5]:
data_mixed = [1, 'abs']

It will be translated to a list in R

In [6]:
%get data_mixed
data_mixed

[[1]]
[1] 1

[[2]]
[1] "abs"

In [7]:
class(data_mixed)

[1] "list"

Similarly, you can get a `data.frame` `mtcars` from R in SoS, but an option `--from R` is needed to specify the source kernel

In [8]:
%get mtcars --from R
mtcars

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
Mazda RX4,21.0,6.0,160.0,110.0,3.90,2.620,16.46,0.0,1.0,4.0,4.0
Mazda RX4 Wag,21.0,6.0,160.0,110.0,3.90,2.875,17.02,0.0,1.0,4.0,4.0
Datsun 710,22.8,4.0,108.0,93.0,3.85,2.320,18.61,1.0,1.0,4.0,1.0
Hornet 4 Drive,21.4,6.0,258.0,110.0,3.08,3.215,19.44,1.0,0.0,3.0,1.0
Hornet Sportabout,18.7,8.0,360.0,175.0,3.15,3.440,17.02,0.0,0.0,3.0,2.0
Valiant,18.1,6.0,225.0,105.0,2.76,3.460,20.22,1.0,0.0,3.0,1.0
Duster 360,14.3,8.0,360.0,245.0,3.21,3.570,15.84,0.0,0.0,3.0,4.0
Merc 240D,24.4,4.0,146.7,62.0,3.69,3.190,20.00,1.0,0.0,4.0,2.0
Merc 230,22.8,4.0,140.8,95.0,3.92,3.150,22.90,1.0,0.0,4.0,2.0
Merc 280,19.2,6.0,167.6,123.0,3.92,3.440,18.30,1.0,0.0,4.0,4.0


The type of `mtcars` in SoS (Python) is, not surprisingly, a Pandas DataFrame

In [9]:
type(mtcars)

pandas.core.frame.DataFrame

You can also `%get` variables from one subkernel in another subkernel. For example the following cell gets `mtcars` from a `Julia` kernel. As the warning message says, because Julia dataframe does not yet support row labels, `mtcars` in Julia will not have row label.

In [10]:
%get mtcars --from R
mtcars

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
Mazda RX4,21.0,6.0,160.0,110.0,3.90,2.620,16.46,0.0,1.0,4.0,4.0
Mazda RX4 Wag,21.0,6.0,160.0,110.0,3.90,2.875,17.02,0.0,1.0,4.0,4.0
Datsun 710,22.8,4.0,108.0,93.0,3.85,2.320,18.61,1.0,1.0,4.0,1.0
Hornet 4 Drive,21.4,6.0,258.0,110.0,3.08,3.215,19.44,1.0,0.0,3.0,1.0
Hornet Sportabout,18.7,8.0,360.0,175.0,3.15,3.440,17.02,0.0,0.0,3.0,2.0
Valiant,18.1,6.0,225.0,105.0,2.76,3.460,20.22,1.0,0.0,3.0,1.0
Duster 360,14.3,8.0,360.0,245.0,3.21,3.570,15.84,0.0,0.0,3.0,4.0
Merc 240D,24.4,4.0,146.7,62.0,3.69,3.190,20.00,1.0,0.0,4.0,2.0
Merc 230,22.8,4.0,140.8,95.0,3.92,3.150,22.90,1.0,0.0,4.0,2.0
Merc 280,19.2,6.0,167.6,123.0,3.92,3.440,18.30,1.0,0.0,4.0,4.0


If you really need such information for your analysis in Julia, you will have to transfer it separately,

In [11]:
cars = row.names(mtcars)

In [12]:
%get cars --from R
cars

32-element Array{String,1}:
 "Mazda RX4"        
 "Mazda RX4 Wag"    
 "Datsun 710"       
 "Hornet 4 Drive"   
 "Hornet Sportabout"
 "Valiant"          
 "Duster 360"       
 "Merc 240D"        
 "Merc 230"         
 "Merc 280"         
 "Merc 280C"        
 "Merc 450SE"       
 "Merc 450SL"       
 ⋮                  
 "Toyota Corona"    
 "Dodge Challenger" 
 "AMC Javelin"      
 "Camaro Z28"       
 "Pontiac Firebird" 
 "Fiat X1-9"        
 "Porsche 914-2"    
 "Lotus Europa"     
 "Ford Pantera L"   
 "Ferrari Dino"     
 "Maserati Bora"    
 "Volvo 142E"       

## Putting variable to another kernel using magic `%put`

Magic `%put` is similar to `%get` but it puts variable from the current kernel to another. It by default put variables to SoS but can put to another subkernel with option `--to`.

For example, the following cell puts variable `ncars` to SoS:

In [13]:
%put ncars
ncars = length(cars)

It is important to note here, that although the `%put` magic is specified at the beginning of the cell (as required by SoS), it is actually executed after the cell is executed.

`ncars` is available in SoS after the `%put` magic

In [14]:
ncars

32

Similarly, you can put variables to another kernel using the `--to` option:

In [15]:
%put df --to R

import pandas as pd

df = pd.DataFrame({'num_legs': [2, 4, 8, 0],
                   'num_wings': [2, 0, 0, 0],
                   'num_specimen_seen': [10, 2, 1, 8]},
             index=['falcon', 'dog', 'spider', 'fish'])

and the variable `df` will be available in R

In [16]:
df

,num_legs,num_wings,num_specimen_seen
,<dbl>,<dbl>,<dbl>
falcon,2,2,10
dog,4,0,2
spider,8,0,1
fish,0,0,8


## Evaluating statements in another kernel with input and output variables using `%with` magic

Say during a Python-based data analysis procedure you are in need of a bunch of random numbers, and either you do not have Scipy installed or are more familiar with how R, you can call R as follows

In [17]:
n = 5

In [18]:
%with R --in n --out rn
rn = rnorm(n)

In [19]:
rn

[0.777989112038058,
 -1.09720171824747,
 -0.937071777746947,
 -0.821497681562507,
 -0.651744430567863]

Here the `%with` magic is just a shortcut to

In [20]:
%get n
%put rn
rn = rnorm(n)

but `%with R` magic will appear to be function-call like procedure without changing the cell kernel.

The `%with` magic can also be used from a subkernel and calling statements in SoS or another subkernel. For example, the following cell calls the `head` function of `DataFrame` to get the first few rows of `mtcars`, and return as `data.frame` in R.

In [21]:
%with SoS -i mtcars -o head
head = mtcars.head()

In [22]:
head

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Mazda RX4,21.0,6,160,110,3.90,2.620,16.46,0,1,4,4
Mazda RX4 Wag,21.0,6,160,110,3.90,2.875,17.02,0,1,4,4
Datsun 710,22.8,4,108,93,3.85,2.320,18.61,1,1,4,1
Hornet 4 Drive,21.4,6,258,110,3.08,3.215,19.44,1,0,3,1
Hornet Sportabout,18.7,8,360,175,3.15,3.440,17.02,0,0,3,2


Compare to all other multi-language approaches such as Python's [`rpy2`](https://rpy2.readthedocs.io/en/version_2.8.x/), Julia's [`PyCall`](https://github.com/JuliaPy/PyCall.jl), or MATLAB's python engine, it is important to note that **all statements and datatypes in a SoS environment are native** and therefore easier to work with, **with the disadvantage that your analyses can only be executed in SoS notebook** (not as a standalone script).

## Further reading

* [How to write a language module](language_module.html) if you would like to write a language module for the kernel you rely on